In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larlite import larlite
from ROOT import larutil
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import lardly

In [ ]:
# LOAD THE FILE
HAS_MCTRACK = True

triplet_files = ["test.root"]
triplet_tree  = rt.TChain("larmatchtriplet")
keypoint_tree = rt.TChain("keypointlabels")
ssnet_tree    = rt.TChain("ssnetlabels")
for f in triplet_files:
    triplet_tree.AddFile(f)
    keypoint_tree.AddFile(f)
    ssnet_tree.AddFile(f)
nentries = keypoint_tree.GetEntries()

# if HAS_MCTRACK:
#     ioll = larlite.storage_manager(larlite.storage_manager.kREAD)
# for f in triplet_files:
#     tree.AddFile( f )
#     if HAS_MCTRACK:
#         ioll.add_in_filename( f.replace(".root","_larlite.root") )
# if HAS_MCTRACK:
#     ioll.open()

print("Num entries: ",nentries)

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
#detid = larlite.geo.kICARUS
detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
# DEFINE PARTICLE ID COLORS
particle_id_color = {0:(0,0,0),      # no label
                     1:(255,125,50), # Cosmic
                     2:(0,0,0),      # BNB
                     3:(255,0,0),    # electron
                     4:(0,255,0),    # gamma
                     5:(0,125,125),  # pi0
                     6:(155,0,155),  # Muon
                     7:(255,255,0),  # Kaon
                     8:(255,165,0),  # pion                     
                     9:(0,0,255)}    # proton

particle_id_name = {0:"nolabel",  # no label
                    1:"delta",    # no label
                    2:"nolabel",  # no label
                    3:"electron", # electron
                    4:"gamma",    # gamma
                    5:"pi0",  # pi0
                    6:"muon", # Muon
                    7:"kaon", # Kaon
                    8:"pion", # proton
                    9:"proton"}   # pion

# define some colors for particle types
kp_color_array = np.array( ((255,0,0),    # nu
                            (0,255,0),    # track-start
                            (0,0,255),    # track-end
                            (255,0,255),  # shower
                            (0,255,255),  # shower-michel
                            (255,255,0)),     # shower-delta
                            dtype=np.float64 ) 

In [ ]:
# Get entry data
ENTRY = 0
triplet_tree.GetEntry(ENTRY)
keypoint_tree.GetEntry(ENTRY)
ssnet_tree.GetEntry(ENTRY)
#ioll.go_to(ENTRY)

spacepoint_arrays = []
keypoint_arrays = []
ssnet_arrays = []
print("number of tpc data: ",triplet_tree.triplet_v.size())
for imatchdata in range(triplet_tree.triplet_v.size()):
    matchdata = triplet_tree.triplet_v.at(imatchdata)
    ntriplets = matchdata._triplet_v.size() # needs function
    if ntriplets==0:
        continue
    icryo = matchdata._cryoid # needs function
    itpc  = matchdata._tpcid # needs function
    
    spdata = matchdata.make_spacepoint_charge_array()
    print("spacepoint_t:",spdata['spacepoint_t'].shape)

    keypointdata = keypoint_tree.keypoint_tpclabel_v.at(imatchdata)
    kplabel = keypointdata.get_triplet_score_array(3.0)
    print("kplabel: ",kplabel.shape)
    spdata['keypoint_t'] = kplabel
    
    ssnetdata = ssnet_tree.ssnet_labeldata_v.at(imatchdata)
    ssnetdict = ssnetdata.make_ssnet_arrays()
    print("ssnetdict keys: ",ssnetdict.keys())
    ssnet_arrays.append( ssnetdict )

    #print("keys: ",spdata.keys())
    #print("spacepoint_t:",spdata['spacepoint_t'].shape)
    #print("truetriplet_t:",spdata['truetriplet_t'].shape)
    spacepoint_arrays.append( (icryo,itpc,spdata) )
    
    kpclass_dict = {}
    for ikpclass in range(6):
        kppos = keypointdata.get_keypoint_array(ikpclass)
        print("kppos[class=",ikpclass,"]: ",kppos.shape)
        kppos = kppos[:,4:7]
        kpcolor = kp_color_array[ikpclass]
        zcolor = "rgb(%d,%d,%d)"%(kpcolor[0],kpcolor[1],kpcolor[2])
        kpclass_plot = go.Scatter3d( x=kppos[:,0], y=kppos[:,1], z=kppos[:,2], 
                            mode="markers", name="KP%d"%(ikpclass), 
                            marker={"size":2.0,"opacity":1.0,"color":zcolor})
        kpclass_dict[ikpclass] = kpclass_plot
    keypoint_arrays.append(kpclass_dict)
    
#ev_mctrack = ioll.get_data(larlite.data.kMCTrack,"mcreco")

In [ ]:
# PLOT Keypoint scores
KPTYPE=5
TRUE_ONLY=True
PLOT_KEYPOINTS = True

sp_plots = []
for (icryo,itpc,spdata) in spacepoint_arrays:
    pos = spdata['spacepoint_t']
    truth = spdata['truetriplet_t']
    kplabel = spdata['keypoint_t'][:,KPTYPE]
    if TRUE_ONLY:
        pos = pos[truth==1,:]
        kplabel = kplabel[truth==1]
    
    plot = go.Scatter3d( x=pos[:,0], y=pos[:,1], z=pos[:,2], 
                        mode="markers", name="(%d,%d)"%(icryo,itpc), 
                        marker={"size":1.0,"opacity":0.5,"color":kplabel,"colorscale":"Viridis"} )  
    sp_plots.append(plot)
    
if PLOT_KEYPOINTS:
    for kpdict in keypoint_arrays:
        for iclass in kpdict:
            sp_plots.append( kpdict[iclass] )

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()

In [ ]:
# PLOT SSNET 
TRUE_ONLY=True
PLOT = 'ssnet_label_t'
#PLOT = 'ssnet_topweight_t'
#PLOT = 'ssnet_classweight_t'
#PLOT = 'ssnet_totalweight'


sp_plots = []
for n,(icryo,itpc,spdata) in enumerate(spacepoint_arrays):
    pos = spdata['spacepoint_t']
    truth = spdata['truetriplet_t']
    ssnetdict = ssnet_arrays[n]
    
    zdata = None
    if PLOT in ["ssnet_label_t","ssnet_topweight_t","ssnet_classweight_t"]:
        zdata = ssnetdict[PLOT]
    elif PLOT=='ssnet_totalweight':
        zdata = ssnetdict['ssnet_topweight_t']*ssnetdict['ssnet_classweight_t']
    
    if TRUE_ONLY:
        pos = pos[truth==1,:]
        zdata = zdata[truth==1]
    
    plot = go.Scatter3d( x=pos[:,0], y=pos[:,1], z=pos[:,2], 
                        mode="markers", name="(%d,%d)"%(icryo,itpc), 
                        marker={"size":1.0,"opacity":0.5,"color":zdata,"colorscale":"Viridis"} )  
    sp_plots.append(plot)


axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()